In [53]:
import pandas as pd
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from collections import namedtuple
from google.colab import files
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

##Importamos los CSV a utilizar para armar las matrices

In [54]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [55]:
id='1slJiG-M_oawvkzaqWzZkSmVxxZor6mKQ'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train_features.csv')

id='1TmL2AY_yymiV8zXVvmbqXsIZoexR9sLs'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('test_features.csv')


id='1J0GcaPrz3SDrfuRjJiSGapHjCTCy094x'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('test_limpio.csv')

id='1r9I2BE6oMDpSlQPS9OJIK_d0OW4G6yAG'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train_limpio.csv')

id='1Pk5MK9Hs_kMUT9NotGnOKE0NPra-39YU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train.csv')

id='1GsTM9oLtIV8-Da_fDOFWsQYMpgQ8GOYJ'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('test.csv')

In [56]:
train_features = pd.read_csv('train_features.csv')
test_features = pd.read_csv('test_features.csv')
test_limpio = pd.read_csv('test_limpio.csv')
train_limpio = pd.read_csv('train_limpio.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [57]:
train['text'] = train_limpio['text']
test['text'] = test_limpio['text']

In [76]:
train['text'] = train['text'].fillna(' ')
test['text'] = test['text'].fillna(' ')
train['keyword'] = train['keyword'].fillna(' ')
test['keyword'] = test['keyword'].fillna(' ')

##Funciones para realizar el encoding con Doc2Vec

In [63]:
def tag(lista_texto):
  text_tag = []
  analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')
  for i, text in enumerate(lista_texto):
      words = text.lower().split()
      tags = [i]
      text_tag.append(analyzedDocument(words, tags))
  return text_tag

In [51]:
def doc2vec(lista_taggeada):
  max_epochs = 10
  vec_size = 100
  alpha = 0.025

  model = Doc2Vec(size=vec_size, alpha=alpha, min_alpha=0.00025, min_count=1, dm =1)
    
  model.build_vocab(lista_taggeada)

  for epoch in range(max_epochs):
      model.train(lista_taggeada, total_examples=model.corpus_count, epochs=model.iter)
      model.alpha -= 0.0002
      model.min_alpha = model.alpha
      
  return model

In [52]:
def get_vectors_doc2vec(modelo):
  vectors = []
  for i in range(len(modelo.docvecs)):
    vectors.append(list(modelo.docvecs[i]))
  return vectors

##Doc2vec, para text solo

In [ ]:
train_lista_texto = list(train['text'])

In [ ]:
test_lista_texto = list(test['text'])


Realizamos el encoding para el Train

In [64]:
tex_taggeado = tag(train_lista_texto)
modelo = doc2vec(tex_taggeado)
train_vectors = get_vectors_doc2vec(modelo)

/usr/local/lib/python3.6/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  # This is added back by InteractiveShellApp.init_path()


In [66]:
matriz_doc2vec_train = pd.DataFrame(train_vectors)
matriz_doc2vec_train.to_csv('matriz_train_doc2vec_solo_text.csv', index=False)

In [67]:
files.download('matriz_train_doc2vec_solo_text.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Realizamos el encoding para el Test

In [68]:
tex_taggeado = tag(test_lista_texto)
modelo = doc2vec(tex_taggeado)
test_vectors = get_vectors_doc2vec(modelo)

/usr/local/lib/python3.6/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  # This is added back by InteractiveShellApp.init_path()


In [69]:
matriz_doc2vec_test = pd.DataFrame(test_vectors)
matriz_doc2vec_test.to_csv('matriz_test_doc2vec_solo_text.csv', index=False)

In [70]:
files.download('matriz_test_doc2vec_solo_text.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##DOC2VEC, para text y keyword

In [95]:
train_lista_texto_keyword = list(train['text'] + ' ' + train['keyword'])
test_lista_texto_keyword = list(test['text'] + ' ' + test['keyword'])

Realizamos el encoding para el Train

In [99]:
tex_taggeado = tag(train_lista_texto_keyword)
modelo = doc2vec(tex_taggeado)
train_vectors_tk = get_vectors_doc2vec(modelo)

/usr/local/lib/python3.6/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  # This is added back by InteractiveShellApp.init_path()


In [111]:
matriz_doc2vec_train2 = pd.DataFrame(train_vectors_tk)
matriz_doc2vec_train2.to_csv('matriz_train_doc2vec_text_keyword.csv', index=False)

In [102]:
files.download('matriz_train_doc2vec_text_keyword.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Realizamos el encoding para el Test

In [103]:
tex_taggeado = tag(test_lista_texto_keyword)
modelo = doc2vec(tex_taggeado)
test_vectors_tk = get_vectors_doc2vec(modelo)

/usr/local/lib/python3.6/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  # This is added back by InteractiveShellApp.init_path()


In [106]:
matriz_doc2vec_test2 = pd.DataFrame(test_vectors_tk)
matriz_doc2vec_test2.to_csv('matriz_test_doc2vec_text_keyword.csv', index=False)

In [108]:
files.download('matriz_test_doc2vec_text_keyword.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Matriz doc2vec para text keyword mas agregado de features numericos

In [109]:
numeric_features_test = test_features[['cant_stop_words', 'prom_long_palabra', 'cant_puntuacion', 'cant_apariciones_keyword', 'cant_numeros', 'cant_mayusculas', 'cant_vocales']]
numeric_features_train = train_features[['cant_stop_words', 'prom_long_palabra', 'cant_puntuacion', 'cant_apariciones_keyword', 'cant_numeros', 'cant_mayusculas', 'cant_vocales']]

In [112]:
doc2vec_features_numericos_train = pd.concat([matriz_doc2vec_train2, numeric_features_train], axis=1, sort=False)
doc2vec_features_numericos_test = pd.concat([matriz_doc2vec_test2, numeric_features_test], axis=1, sort=False)

In [115]:
doc2vec_features_numericos_train.to_csv('doc2vec_features_numericos_train.csv', index=False)
doc2vec_features_numericos_test.to_csv('doc2vec_features_numericos_test.csv', index=False)

In [116]:
files.download('doc2vec_features_numericos_train.csv')
files.download('doc2vec_features_numericos_test.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>